In [27]:
import pandas as pd
from itertools import permutations
from numpy import cos, sin, arcsin, sqrt
from math import radians
import requests
import urllib
import time
import json

In [2]:
stations_2014 = pd.read_csv('data/2014/Stations_2014.csv')
stations_2015 = pd.read_csv('data/2015/Stations_2015.csv')
stations_2016 = pd.read_csv('data/2016/Stations_2016.csv')
stations_2017 = pd.read_csv('data/2017/Stations_2017.csv')
stations_2018 = pd.read_csv('data/2018/Stations_2018.csv')
stations_2019 = pd.read_csv('data/2019/Stations_2019.csv')

In [3]:
for station in [stations_2017, stations_2018, stations_2019]:
    print(len(station))

546
552
619


In [4]:
stations_2018

,code,name,latitude,longitude
0,7030,de Bordeaux / Marie-Anne,45.533409,-73.570657
1,6141,de Bordeaux / Rachel,45.532270,-73.568280
2,6100,Mackay / de Maisonneuve,45.496590,-73.578510
3,6064,Métro Peel (de Maisonneuve / Stanley),45.500380,-73.575070
4,6730,35e avenue / Beaubien,45.570081,-73.573047
...,...,...,...,...
547,7014,Métro Université de Montréal,45.503640,-73.618290
548,6928,Jean-Brillant / McKenna,45.498767,-73.618870
549,6084,Duke / Brennan,45.495581,-73.553711
550,7082,Métro Lionel-Groulx (Atwater / Lionel-Groulx),45.482941,-73.579742


In [5]:
station_0 = stations_2019.iloc[:1]

In [6]:
station_0

,Code,name,latitude,longitude
0,10002,Métro Charlevoix (Centre / Charlevoix),45.478228,-73.569651


In [7]:
# density = 0
# for row in stations_2019.iterrows():
#     lon1 = station_0['longitude']
#     lat1 = station_0['latitude']

#     lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, row[1]['longitude'], row[1]['latitude']])
#     dlon = lon2 - lon1 
#     dlat = lat2 - lat1 
#     a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
#     c = 2 * arcsin(sqrt(a)) 
#     km = 6367 * c

#     if (km < 10):
#         print(km)
#         density += 1

# print(density)

In [8]:
# distance_frame = pd.DataFrame(columns=['base_station_code', 'target_station_code', 'distance'])
# density_frame = pd.DataFrame(columns=['station_code', 'density'])
# stations_2019_test = stations_2019.head(10)


# for index in permutations(stations_2019_test.index, 2):

#     base_station_code = stations_2019_test.at[index[0], 'Code']
#     lon1 = stations_2019_test.iloc[[index[0]]]['longitude']
#     lat1 = stations_2019_test.iloc[[index[0]]]['latitude']

#     target_station_code = stations_2019_test.at[index[1], 'Code']
#     lon2 = stations_2019_test.iloc[[index[1]]]['longitude']
#     lat2 = stations_2019_test.iloc[[index[1]]]['latitude']

#     lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
#     dlon = lon2 - lon1 
#     dlat = lat2 - lat1 
#     a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
#     c = 2 * arcsin(sqrt(a)) 
#     km = 6367 * c

#     distance_frame = distance_frame.append({
#         'base_station_code': base_station_code,
#         'target_station_code':target_station_code,
#         'distance': km
#     }, ignore_index=True)

#     distance_frame[['base_station_code', 'target_station_code']] = distance_frame[['base_station_code', 'target_station_code']].astype('int')

# unique_base_station_codes = list(distance_frame.base_station_code.unique())

# for base_station_code in unique_base_station_codes:

#     dict = {
#         'station_code': base_station_code,
#         'density': len(distance_frame[(distance_frame['base_station_code']==base_station_code) & (distance_frame['distance'] < 7)])
#     }

#     df2 = pd.DataFrame([dict])

#     density_frame = pd.concat([density_frame, df2])

# stations_2019_test = stations_2019_test.merge(density_frame, left_on='Code', right_on='station_code').drop(columns='station_code')

# stations_2019_test

In [9]:
def calculate_station_distances(station_df, threshhold):
    distance_frame = pd.DataFrame(columns=['base_station_code', 'target_station_code', 'distance'])
    i = 0
    distance_dict = {}

    for index in permutations(station_df.index, 2):

        base_station_code = station_df.at[index[0], 'code']
        lon1 = station_df.iloc[[index[0]]]['longitude']
        lat1 = station_df.iloc[[index[0]]]['latitude']

        target_station_code = station_df.at[index[1], 'code']
        lon2 = station_df.iloc[[index[1]]]['longitude']
        lat2 = station_df.iloc[[index[1]]]['latitude']

        # calculate distance
        lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
        dlon = lon2 - lon1 
        dlat = lat2 - lat1 
        a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
        c = 2 * arcsin(sqrt(a)) 
        km = 6367 * c

        if km > threshhold:
            continue
        else:
            distance_dict[i] = {
            'base_station_code': base_station_code,
            'target_station_code':target_station_code,
            'distance': km
        }
            i += 1
    
    distance_frame = pd.DataFrame.from_dict(distance_dict, "index")

    distance_frame[['base_station_code', 'target_station_code']] = distance_frame[['base_station_code', 'target_station_code']].astype('int')

    distance_frame = distance_frame[distance_frame['distance'] < threshhold]


    
    return distance_frame

In [10]:
distance_frame = calculate_station_distances(stations_2014, 1)

In [11]:
distance_frame

,base_station_code,target_station_code,distance
0,6209,6214,0.550309
1,6209,6211,0.523232
2,6209,6206,0.434329
3,6209,6411,0.202936
4,6209,6020,0.482152
...,...,...,...
10815,6178,6143,0.641299
10816,6178,6188,0.412111
10817,6178,6190,0.749158
10818,6178,6044,0.550707


In [12]:
def get_processed_stations_density_frame(distance_frame):
    unique_base_station_codes = list(distance_frame.base_station_code.unique())

    i = 0
    density_dict = {}

    for base_station_code in unique_base_station_codes:

        density_dict[i] = {
            'station_code': base_station_code,
            'density': len(distance_frame[(distance_frame['base_station_code']==base_station_code)])
        }
        i+=1


    density_frame = pd.DataFrame.from_dict(density_dict, "index")

    stations_processed = stations_2019.merge(density_frame, left_on='Code', right_on='station_code', how='inner').drop(columns='station_code')

    return stations_processed

In [13]:
stations_processed = get_processed_stations_density_frame(distance_frame)

In [14]:
# stations_2019_processed = stations_2019.merge(density_frame, left_on='Code', right_on='station_code', how='inner').drop(columns='station_code')

In [15]:
stations_processed

,Code,name,latitude,longitude,density
0,10002,Métro Charlevoix (Centre / Charlevoix),45.478228,-73.569651,12
1,5002,St-Charles / Montarville,45.533682,-73.515261,3
2,5003,Place Longueuil,45.528410,-73.517166,3
3,5004,St-Charles / Grant,45.539824,-73.508752,2
4,5005,St-Charles / St-Jean,45.536907,-73.511914,3
...,...,...,...,...,...
450,6925,des Écores / Jean-Talon,45.552364,-73.602868,14
451,6926,Marie-Anne / St-Hubert,45.524683,-73.578897,46
452,6927,Édouard-Montpetit / de Stirling,45.507230,-73.615085,7
453,6928,Jean-Brillant / McKenna,45.498767,-73.618870,5


In [16]:
stations_processed.describe()

,Code,latitude,longitude,density
count,455.000000,455.000000,455.000000,455.000000
mean,6300.692308,45.519988,-73.582562,23.573626
std,328.050929,0.027359,0.027445,14.494058
min,5002.000000,45.417147,-73.667162,1.000000
25%,6121.500000,45.502010,-73.599254,11.000000
50%,6241.000000,45.523492,-73.577615,23.000000
75%,6395.500000,45.538838,-73.564786,36.000000
max,10002.000000,45.640003,-73.490113,51.000000


In [37]:
url = r'https://api.open-elevation.com/api/v1/lookup?'

def elevation_function(df, lat_column, lon_column):
    elevations = []
    for lat, lon in zip(df[lat_column], df[lon_column]):
        
        coords = str(lat) + "," + str(lon)

        params = {
            'locations': coords,
        }
        
        result = requests.get((url + urllib.parse.urlencode(params)))
        print(result)
        elevations.append(result.json()['results'][0]['elevation'])
        time.sleep(0.2)

    df['elevation_meters'] = elevations

In [38]:
elevation_function(stations_processed, 'latitude', 'longitude')

In [39]:
stations_processed

,Code,name,latitude,longitude,density,elevation_meters
0,10002,Métro Charlevoix (Centre / Charlevoix),45.478228,-73.569651,12,16
1,5002,St-Charles / Montarville,45.533682,-73.515261,3,16
2,5003,Place Longueuil,45.528410,-73.517166,3,15
3,5004,St-Charles / Grant,45.539824,-73.508752,2,13
4,5005,St-Charles / St-Jean,45.536907,-73.511914,3,16
...,...,...,...,...,...,...
450,6925,des Écores / Jean-Talon,45.552364,-73.602868,14,52
451,6926,Marie-Anne / St-Hubert,45.524683,-73.578897,46,51
452,6927,Édouard-Montpetit / de Stirling,45.507230,-73.615085,7,111
453,6928,Jean-Brillant / McKenna,45.498767,-73.618870,5,117


In [ ]:
stations_processed.to_csv('data/stations_preprocessed/Stations_2014.csv')